In [1]:
import sys
from pathlib import Path

# Add the parent directory of the notebook to the Python path
sys.path.append(str(Path().resolve().parent))

# Notes:
- It does not makes sense to aggregate order book only to drop select

In [11]:
import pandas as pd

In [12]:
from funcs.data_read import (
    read_txn_data, 
    read_ob_data,
    #preprocess_txn_data, 
    #compute_lob_features, 
    #create_lob_dataset, 
    #merge_txn_and_lob
)

In [13]:
df=read_ob_data(test=True,test_size=2)
df.reset_index(drop=True)

Processing 2 file(s): ['C:\\Users\\peter\\codes\\ml\\eth\\Komplex\\data\\bitstamp_ob_04_07_2018.zip', 'C:\\Users\\peter\\codes\\ml\\eth\\Komplex\\data\\bitstamp_ob_04_08_2018.zip']
Processing bitstamp_ob_04_07_2018.zip...
Processing bitstamp_ob_04_08_2018.zip...
Finished loading. Total rows: 49,188,752


,type,price,amount,datetime
0,b,6613.32,0.068823,2018-07-03 22:00:01+00:00
1,b,6613.31,3.709311,2018-07-03 22:00:01+00:00
2,b,6608.79,0.015000,2018-07-03 22:00:01+00:00
3,b,6608.78,1.000000,2018-07-03 22:00:01+00:00
4,b,6608.77,1.911835,2018-07-03 22:00:01+00:00
...,...,...,...,...
49188747,a,15217.44,0.001151,2018-08-04 21:59:34+00:00
49188748,a,15221.48,0.005494,2018-08-04 21:59:34+00:00
49188749,a,15225.00,0.001960,2018-08-04 21:59:34+00:00
49188750,a,15227.00,0.100600,2018-08-04 21:59:34+00:00


In [14]:
test=df.iloc[:1000000].copy()

In [15]:
test

,type,price,amount,datetime
0,b,6613.32,0.068823,2018-07-03 22:00:01+00:00
1,b,6613.31,3.709311,2018-07-03 22:00:01+00:00
2,b,6608.79,0.015000,2018-07-03 22:00:01+00:00
3,b,6608.78,1.000000,2018-07-03 22:00:01+00:00
4,b,6608.77,1.911835,2018-07-03 22:00:01+00:00
...,...,...,...,...
999995,a,7675.22,0.010000,2018-07-03 22:59:34+00:00
999996,a,7677.00,0.160000,2018-07-03 22:59:34+00:00
999997,a,7678.61,0.003006,2018-07-03 22:59:34+00:00
999998,a,7680.00,1.236824,2018-07-03 22:59:34+00:00


In [16]:
# Convert datetime to Unix timestamp (seconds since epoch)
test['timestamp_id'] = (test['datetime'].astype('int64') // 10**9).astype(int)
test

,type,price,amount,datetime,timestamp_id
0,b,6613.32,0.068823,2018-07-03 22:00:01+00:00,1530655201
1,b,6613.31,3.709311,2018-07-03 22:00:01+00:00,1530655201
2,b,6608.79,0.015000,2018-07-03 22:00:01+00:00,1530655201
3,b,6608.78,1.000000,2018-07-03 22:00:01+00:00,1530655201
4,b,6608.77,1.911835,2018-07-03 22:00:01+00:00,1530655201
...,...,...,...,...,...
999995,a,7675.22,0.010000,2018-07-03 22:59:34+00:00,1530658774
999996,a,7677.00,0.160000,2018-07-03 22:59:34+00:00,1530658774
999997,a,7678.61,0.003006,2018-07-03 22:59:34+00:00,1530658774
999998,a,7680.00,1.236824,2018-07-03 22:59:34+00:00,1530658774


In [50]:
ask_amt=test[(test["timestamp_id"] == 1530655201) & (test["type"] == "b")]["amount"].sum()
bid_amt=test[(test["timestamp_id"] == 1530655201) & (test["type"] == "a")]["amount"].sum()
print(ask_amt/bid_amt)

226.67011750672162


# Basic Assimetry

In [103]:
# Group by timestamp_id and calculate metrics
order_book_snapshot = test.groupby('timestamp_id').apply(
    lambda x: pd.Series({
        'highest_bid': x[x['type'] == 'b']['price'].max(),
        'lowest_ask': x[x['type'] == 'a']['price'].min(),
        'total_bid_volume': x[x['type'] == 'b']['amount'].sum(),
        'total_ask_volume': x[x['type'] == 'a']['amount'].sum(),
        'mid_price': (x[x['type'] == 'b']['price'].max() + x[x['type'] == 'a']['price'].min()) / 2,
        'spread': x[x['type'] == 'a']['price'].min() - x[x['type'] == 'b']['price'].max()
    })
).reset_index()

# Add datetime column back (all rows in group have same datetime)
timestamps = test.drop_duplicates('timestamp_id').set_index('timestamp_id')['datetime']
order_book_snapshot['datetime'] = order_book_snapshot['timestamp_id'].map(timestamps)

# Reorder columns
order_book_snapshot = order_book_snapshot[['timestamp_id', 'datetime', 
                                          'highest_bid', 'lowest_ask', 
                                          'mid_price', 'spread',
                                          'total_bid_volume', 'total_ask_volume']]

In [104]:
order_book_snapshot

,timestamp_id,datetime,highest_bid,lowest_ask,mid_price,spread,total_bid_volume,total_ask_volume
0,1530655201,2018-07-03 22:00:01+00:00,6613.32,6614.99,6614.155,1.67,567123.997426,2501.979545
1,1530655234,2018-07-03 22:00:34+00:00,6613.32,6614.99,6614.155,1.67,567125.105659,2500.004667
2,1530655261,2018-07-03 22:01:01+00:00,6613.32,6614.99,6614.155,1.67,567127.754102,2495.362910
3,1530655294,2018-07-03 22:01:34+00:00,6608.82,6614.98,6611.900,6.16,567127.251110,2495.541990
4,1530655321,2018-07-03 22:02:01+00:00,6608.80,6614.25,6611.525,5.45,567128.324518,2505.468060
...,...,...,...,...,...,...,...,...
115,1530658654,2018-07-03 22:57:34+00:00,6472.89,6482.28,6477.585,9.39,567289.041076,2623.621335
116,1530658682,2018-07-03 22:58:02+00:00,6468.77,6479.10,6473.935,10.33,567295.605745,2608.720942
117,1530658714,2018-07-03 22:58:34+00:00,6479.73,6481.57,6480.650,1.84,567305.772930,2609.622702
118,1530658741,2018-07-03 22:59:01+00:00,6476.16,6481.38,6478.770,5.22,567316.741079,2658.871152


In [ ]:
order_book_snapshot["bid_ask_volume_ratio"]=order_book_snapshot["total_bid_volume"]/order_book_snapshot["total_ask_volume"]
#order_book_snapshot.loc[order_book_snapshot["bid_ask_volume_ratio"].idxmax()]
order_book_snapshot.loc[order_book_snapshot["total_bid_volume"].idxmax()]
order_book_snapshot.loc[order_book_snapshot["total_ask_volume"].idxmax()]

# IF I AM WILLING TO Pay X PERCENT MORE HOW MUCH % more i get from the oders?

In [ ]:
def calculate_percentage_ranges(group):
    bids = group[group['type'] == 'b']
    asks = group[group['type'] == 'a']
    
    highest_bid = bids['price'].max()
    lowest_ask = asks['price'].min()
    mid_price = (highest_bid + lowest_ask) / 2
    
    # Initialize result dictionary
    result = {
        'highest_bid': highest_bid,
        'lowest_ask': lowest_ask,
        'mid_price': mid_price,
        'spread': lowest_ask - highest_bid,
        'total_bid_volume': bids['amount'].sum(),
        'total_ask_volume': asks['amount'].sum()
    }
    
    # Define percentage ranges (10% to 100% in 10% increments)
    percentages = range(10, 101, 10)
    
    # Calculate for asks (price > mid_price)
    for p in percentages:
        upper = mid_price * (1 + p/100)
        lower = mid_price if p == 10 else mid_price * (1 + (p-10)/100)
        #mask = (asks['price'] >= lower) & (asks['price'] < upper)
        mask = (asks['price'] < upper)
        #result[f'ask_p{p}'] = asks[mask]['amount'].sum() / result['total_ask_volume'] if result['total_ask_volume'] > 0 else 0
        result[f'ask_p{p}'] = asks[mask]['amount'].sum() if result['total_ask_volume'] > 0 else 0

    # Calculate for bids (price < mid_price)
    for p in percentages:
        upper = mid_price if p == 10 else mid_price * (1 - (p-10)/100)
        lower = mid_price * (1 - p/100)
        #mask = (bids['price'] > lower) & (bids['price'] <= upper)
        mask = (bids['price'] > lower)
        #result[f'bid_p{p}'] = bids[mask]['amount'].sum() / result['total_bid_volume'] if result['total_bid_volume'] > 0 else 0
        result[f'bid_p{p}'] = bids[mask]['amount'].sum() if result['total_bid_volume'] > 0 else 0
  
    return pd.Series(result)

# Apply to grouped data
order_book_snapshot = test.groupby('timestamp_id').apply(calculate_percentage_ranges).reset_index()

# Add datetime back
timestamps = test.drop_duplicates('timestamp_id').set_index('timestamp_id')['datetime']
order_book_snapshot['datetime'] = order_book_snapshot['timestamp_id'].map(timestamps)

# Reorder columns for better readability
cols = ['timestamp_id', 'datetime', 'mid_price', 'highest_bid', 'lowest_ask', 'spread', 
        'total_bid_volume', 'total_ask_volume']

cols += [f'bid_p{p}' for p in range(10, 101, 10)]
cols += [f'ask_p{p}' for p in range(10, 101, 10)]
order_book_snapshot = order_book_snapshot[cols]

In [ ]:
# the percatage of the total volume amout is 
order_book_snapshot.iloc[:, 8:18].div(order_book_snapshot["total_bid_volume"], axis=0)

,bid_p10,bid_p20,bid_p30,bid_p40,bid_p50,bid_p60,bid_p70,bid_p80,bid_p90,bid_p100
0,0.002975,0.007544,0.012659,0.016213,0.019571,0.024365,0.026003,0.027134,0.030839,1.0
1,0.002977,0.007546,0.012661,0.016215,0.019573,0.024367,0.026005,0.027136,0.030841,1.0
2,0.002981,0.007551,0.012665,0.016219,0.019578,0.024371,0.026010,0.027140,0.030846,1.0
3,0.002981,0.007595,0.012665,0.016219,0.019768,0.024371,0.026009,0.027140,0.030845,1.0
4,0.002987,0.007597,0.012666,0.016220,0.019770,0.024372,0.026011,0.027141,0.030847,1.0
...,...,...,...,...,...,...,...,...,...,...
115,0.003661,0.007920,0.012847,0.016879,0.020306,0.024868,0.026332,0.027551,0.031129,1.0
116,0.003671,0.007932,0.012859,0.016893,0.020317,0.024879,0.026343,0.027563,0.031140,1.0
117,0.003685,0.007949,0.012875,0.016907,0.020335,0.024897,0.026360,0.027580,0.031157,1.0
118,0.003704,0.007968,0.012894,0.016926,0.020354,0.024915,0.026379,0.027599,0.031176,1.0


In [109]:
order_book_snapshot.iloc[:, 18:28].div(order_book_snapshot["total_ask_volume"], axis=0)

,ask_p10,ask_p20,ask_p30,ask_p40,ask_p50,ask_p60,ask_p70,ask_p80,ask_p90,ask_p100
0,0.402869,0.505119,0.552025,0.605443,0.684821,0.768820,0.818827,0.887929,0.941991,0.962464
1,0.407290,0.509621,0.556564,0.610024,0.689466,0.773530,0.823577,0.892733,0.946838,0.967328
2,0.406175,0.508696,0.555727,0.609287,0.688875,0.773096,0.823236,0.892521,0.946727,0.967254
3,0.406214,0.508732,0.555613,0.609314,0.668656,0.773113,0.821246,0.892516,0.946516,0.967235
4,0.403936,0.506031,0.552544,0.606235,0.665342,0.769385,0.817287,0.888314,0.941741,0.962738
...,...,...,...,...,...,...,...,...,...,...
115,0.419092,0.524306,0.565790,0.619283,0.672417,0.772569,0.827706,0.875695,0.944543,0.964637
116,0.415344,0.521186,0.562772,0.616684,0.670173,0.770427,0.826231,0.874550,0.943800,0.963778
117,0.414251,0.520027,0.562048,0.616216,0.668931,0.769624,0.825052,0.873342,0.942516,0.962718
118,0.426806,0.530631,0.571560,0.624344,0.676775,0.775597,0.830004,0.877357,0.945291,0.965123


In [82]:
#order_book_snapshot.iloc[:,8:18].sum(axis=1)
#order_book_snapshot.iloc[:,8:18]
order_book_snapshot.iloc[:,18:]

,ask_p10,ask_p20,ask_p30,ask_p40,ask_p50,ask_p60,ask_p70,ask_p80,ask_p90,ask_p100
0,0.402869,0.505119,0.552025,0.605443,0.684821,0.768820,0.818827,0.887929,0.941991,0.962464
1,0.407290,0.509621,0.556564,0.610024,0.689466,0.773530,0.823577,0.892733,0.946838,0.967328
2,0.406175,0.508696,0.555727,0.609287,0.688875,0.773096,0.823236,0.892521,0.946727,0.967254
3,0.406214,0.508732,0.555613,0.609314,0.668656,0.773113,0.821246,0.892516,0.946516,0.967235
4,0.403936,0.506031,0.552544,0.606235,0.665342,0.769385,0.817287,0.888314,0.941741,0.962738
...,...,...,...,...,...,...,...,...,...,...
115,0.419092,0.524306,0.565790,0.619283,0.672417,0.772569,0.827706,0.875695,0.944543,0.964637
116,0.415344,0.521186,0.562772,0.616684,0.670173,0.770427,0.826231,0.874550,0.943800,0.963778
117,0.414251,0.520027,0.562048,0.616216,0.668931,0.769624,0.825052,0.873342,0.942516,0.962718
118,0.426806,0.530631,0.571560,0.624344,0.676775,0.775597,0.830004,0.877357,0.945291,0.965123


# If I would buy more how much more (ON AVERAGE) would i pay if I would take X percent of the oders?

In [ ]:
def calculate_price_differences(group):
    bids = group[group['type'] == 'b']
    asks = group[group['type'] == 'a']
    
    highest_bid = bids['price'].max()
    lowest_ask = asks['price'].min()
    mid_price = (highest_bid + lowest_ask) / 2
    
    # Initialize result dictionary
    result = {
        'highest_bid': highest_bid,
        'lowest_ask': lowest_ask,
        'mid_price': mid_price,
        'spread': lowest_ask - highest_bid,
        'total_bid_volume': bids['amount'].sum(),
        'total_ask_volume': asks['amount'].sum()
    }
    
    # Define percentage ranges (10% to 100% in 10% increments)
    percentages = [10,20,30,40,50,60,70,80,90,100]
    
    # Calculate for asks (price > mid_price)
    ask_cumulative = 0
    sorted_asks = asks.sort_values('price')  # Sort asks from lowest to highest
    
    for p in percentages:
        target_volume = result['total_ask_volume'] * p / 100
        current_volume = 0
        avg_price_diff = 0
        
        for _, row in sorted_asks.iterrows():
            if current_volume >= target_volume:
                break
            volume_to_add = min(row['amount'], target_volume - current_volume)
            avg_price_diff += volume_to_add * (row['price'] - mid_price)
            current_volume += volume_to_add
        
        if current_volume > 0:
            avg_price_diff = avg_price_diff / current_volume
        else:
            avg_price_diff = 0
            
        result[f'ask_p{p}_price_diff'] = avg_price_diff
    
    # Calculate for bids (price < mid_price)
    bid_cumulative = 0
    sorted_bids = bids.sort_values('price', ascending=False)  # Sort bids from highest to lowest
    
    for p in percentages:
        target_volume = result['total_bid_volume'] * p / 100
        current_volume = 0
        avg_price_diff = 0
        
        for _, row in sorted_bids.iterrows():
            if current_volume >= target_volume:
                break
            volume_to_add = min(row['amount'], target_volume - current_volume)
            avg_price_diff += volume_to_add * (mid_price - row['price'])  # How much less you get vs mid
            current_volume += volume_to_add
        
        #if current_volume > 0:
        #    avg_price_diff = avg_price_diff / current_volume
        #else:
        #    avg_price_diff = 0
            
        result[f'bid_p{p}_price_diff'] = avg_price_diff
    
    return pd.Series(result)

# Apply to grouped data
order_book_snapshot = test.groupby('timestamp_id').apply(calculate_price_differences).reset_index()

# Add datetime back
timestamps = test.drop_duplicates('timestamp_id').set_index('timestamp_id')['datetime']
order_book_snapshot['datetime'] = order_book_snapshot['timestamp_id'].map(timestamps)

# Reorder columns for better readability
cols = ['timestamp_id', 'datetime', 'mid_price', 'highest_bid', 'lowest_ask', 'spread', 
        'total_bid_volume', 'total_ask_volume']
cols += [f'bid_p{p}_price_diff' for p in range(10, 101, 10)]
cols += [f'ask_p{p}_price_diff' for p in range(10, 101, 10)]
order_book_snapshot = order_book_snapshot[cols]

In [102]:

#order_book_snapshot.iloc[:, 8:18].div(order_book_snapshot["total_bid_volume"], axis=0)
order_book_snapshot.iloc[:,8:]



,bid_p10_price_diff,bid_p20_price_diff,bid_p30_price_diff,bid_p40_price_diff,bid_p50_price_diff,bid_p60_price_diff,bid_p70_price_diff,bid_p80_price_diff,bid_p90_price_diff,bid_p100_price_diff,ask_p10_price_diff,ask_p20_price_diff,ask_p30_price_diff,ask_p40_price_diff,ask_p50_price_diff,ask_p60_price_diff,ask_p70_price_diff,ask_p80_price_diff,ask_p90_price_diff,ask_p100_price_diff
0,5377.407460,5995.521823,6201.699548,6304.795080,6366.663058,6407.909610,6437.371808,6459.468457,6476.654740,6490.403766,24.727964,52.449765,96.113761,172.135753,309.756001,577.360482,935.782175,1286.075043,1688.263693,2144.113161
1,5377.281590,5995.458880,6201.657587,6304.763608,6366.637881,6407.888629,6437.353825,6459.452722,6476.640753,6490.391177,23.247830,50.000931,92.917757,165.995164,299.222303,558.522468,914.199668,1262.000979,1660.835663,2109.709599
2,5376.979563,5995.307848,6201.556899,6304.688092,6366.577468,6407.838284,6437.310672,6459.414963,6476.607190,6490.360971,23.950500,51.022904,94.113305,167.891275,302.026364,562.710235,918.639982,1266.369821,1665.114081,2113.727227
3,5374.794963,5993.088052,6199.325368,6302.450694,6364.336549,6405.595019,6435.065731,6457.168764,6474.360013,6488.113011,23.964223,51.648400,95.262453,169.277946,303.536874,564.273106,920.269231,1268.058580,1666.850466,2115.508980
4,5374.297767,5992.651947,6198.909631,6302.045141,6363.937107,6405.199651,6434.673272,6456.778488,6473.971434,6487.725790,22.881958,51.461542,95.844236,171.601421,308.601595,574.891253,933.159887,1283.449545,1685.658826,2141.425145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,5233.872661,5855.468306,6062.807204,6166.483289,6228.699622,6270.178411,6299.806495,6322.027558,6339.310607,6353.137046,41.012760,85.134200,146.614545,213.579733,331.917184,552.630579,899.683916,1250.492088,1651.615409,2104.400360
116,5229.536905,5851.475383,6058.928589,6162.661826,6224.912452,6266.414102,6296.058516,6318.291827,6335.584401,6349.418461,41.777150,89.657526,152.455103,221.703428,342.294116,568.078616,917.040315,1267.869763,1668.938348,2121.429134
117,5235.090271,5857.609498,6065.256332,6169.086382,6231.395096,6272.935472,6302.607547,6324.861604,6342.170315,6356.017283,36.481251,84.506313,147.172560,216.962011,338.265163,566.420777,916.568490,1268.043958,1670.080706,2124.628792
118,5231.984320,5855.116448,6062.967632,6166.899854,6229.269874,6270.851120,6300.552389,6322.828340,6340.154080,6354.014672,33.108548,71.298233,133.329268,198.028275,312.300017,523.455942,866.875910,1218.384490,1620.433982,2075.168860


# How much more would i pay for the LAST unit

In [127]:
def calculate_marginal_price_differences(group):
    bids = group[group['type'] == 'b']
    asks = group[group['type'] == 'a']
    
    highest_bid = bids['price'].max() if not bids.empty else 0
    lowest_ask = asks['price'].min() if not asks.empty else 0
    mid_price = (highest_bid + lowest_ask) / 2 if (not bids.empty) and (not asks.empty) else 0
    
    result = {
        'highest_bid': highest_bid,
        'lowest_ask': lowest_ask,
        'mid_price': mid_price,
        'spread': lowest_ask - highest_bid,
        'total_bid_volume': bids['amount'].sum() if not bids.empty else 0,
        'total_ask_volume': asks['amount'].sum() if not asks.empty else 0
    }
    
    percentages = [1,2,3,4,5,10,30,70,100]
    
    # Initialize all columns (even if volume is zero)
    for p in percentages:
        result[f'bid_p{p}_marginal_diff'] = 0  # Default to 0 if no volume
        result[f'ask_p{p}_marginal_diff'] = 0
    
    # Calculate for asks (only if volume exists)
    if not asks.empty:
        sorted_asks = asks.sort_values('price')
        prev_volume = 0
        for p in percentages:
            target_volume = result['total_ask_volume'] * p / 100
            current_volume = 0
            marginal_price_diff = 0
            
            for _, row in sorted_asks.iterrows():
                if current_volume >= target_volume:
                    break
                volume_to_add = min(row['amount'], target_volume - current_volume)
                marginal_price_diff = (row['price'] - mid_price)
                current_volume += volume_to_add
            
            if current_volume > prev_volume:
                result[f'ask_p{p}_marginal_diff'] = marginal_price_diff
            prev_volume = current_volume
    
    # Calculate for bids (only if volume exists)
    if not bids.empty:
        sorted_bids = bids.sort_values('price', ascending=False)
        prev_volume = 0
        for p in percentages:
            target_volume = result['total_bid_volume'] * p / 100
            current_volume = 0
            marginal_price_diff = 0
            
            for _, row in sorted_bids.iterrows():
                if current_volume >= target_volume:
                    break
                volume_to_add = min(row['amount'], target_volume - current_volume)
                marginal_price_diff = (mid_price - row['price'])
                current_volume += volume_to_add
            
            if current_volume > prev_volume:
                result[f'bid_p{p}_marginal_diff'] = marginal_price_diff
            prev_volume = current_volume
    
    return pd.Series(result)

In [128]:
order_book_snapshot_volume_impact = test.groupby('timestamp_id').apply(calculate_marginal_price_differences).reset_index()

In [130]:
order_book_snapshot_volume_impact

,timestamp_id,highest_bid,lowest_ask,mid_price,spread,total_bid_volume,total_ask_volume,bid_p1_marginal_diff,ask_p1_marginal_diff,bid_p2_marginal_diff,...,bid_p5_marginal_diff,ask_p5_marginal_diff,bid_p10_marginal_diff,ask_p10_marginal_diff,bid_p30_marginal_diff,ask_p30_marginal_diff,bid_p70_marginal_diff,ask_p70_marginal_diff,bid_p100_marginal_diff,ask_p100_marginal_diff
0,1530655201,6613.32,6614.99,6614.155,1.67,567123.997426,2501.979545,1614.155,7.515,3364.155,...,6514.155,24.235,6613.155,48.285,6614.055,246.305,6614.145,3371.845,6614.145,7392.105
1,1530655234,6613.32,6614.99,6614.155,1.67,567125.105659,2500.004667,1614.155,7.505,3364.155,...,6514.155,20.285,6613.155,44.735,6614.055,229.845,6614.145,3359.845,6614.145,7380.585
2,1530655261,6613.32,6614.99,6614.155,1.67,567127.754102,2495.362910,1614.155,6.995,3364.155,...,6514.155,23.665,6613.155,44.735,6614.055,233.845,6614.145,3359.845,6614.145,7382.515
3,1530655294,6608.82,6614.98,6611.900,6.16,567127.251110,2495.541990,1611.900,5.480,3361.900,...,6511.900,19.460,6610.900,46.050,6611.800,236.100,6611.890,3362.100,6611.890,7383.100
4,1530655321,6608.80,6614.25,6611.525,5.45,567128.324518,2505.468060,1611.525,4.035,3361.525,...,6511.525,19.495,6610.525,44.105,6611.425,238.475,6611.515,3374.475,6611.515,7388.865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1530658654,6472.89,6482.28,6477.585,9.39,567289.041076,2623.621335,1477.585,15.915,3171.585,...,6377.585,38.525,6476.585,69.845,6477.485,322.415,6477.575,3442.415,6477.575,7429.415
116,1530658682,6468.77,6479.10,6473.935,10.33,567295.605745,2608.720942,1473.935,13.345,3167.935,...,6373.935,42.175,6472.935,73.495,6473.835,326.065,6473.925,3446.065,6473.925,7461.065
117,1530658714,6479.73,6481.57,6480.650,1.84,567305.772930,2609.622702,1480.650,12.510,3174.650,...,6380.650,35.460,6479.650,66.780,6480.550,319.350,6480.640,3439.350,6480.640,7458.330
118,1530658741,6476.16,6481.38,6478.770,5.22,567316.741079,2658.871152,1478.770,14.690,3172.770,...,6378.770,31.170,6477.770,62.540,6478.670,321.230,6478.760,3441.230,6478.760,7421.230


In [129]:
order_book_snapshot_volume_impact.iloc[:, 8:18].div((order_book_snapshot_volume_impact)["mid_price"], axis=0)

,ask_p1_marginal_diff,bid_p2_marginal_diff,ask_p2_marginal_diff,bid_p3_marginal_diff,ask_p3_marginal_diff,bid_p4_marginal_diff,ask_p4_marginal_diff,bid_p5_marginal_diff,ask_p5_marginal_diff,bid_p10_marginal_diff
0,0.001136,0.508630,0.001502,0.848809,0.002167,0.984881,0.002550,0.984881,0.003664,0.999849
1,0.001135,0.508630,0.001502,0.848809,0.002049,0.984881,0.002412,0.984881,0.003067,0.999849
2,0.001058,0.508630,0.001502,0.848809,0.002049,0.984881,0.002412,0.984881,0.003578,0.999849
3,0.000829,0.508462,0.001478,0.848758,0.001950,0.984876,0.002754,0.984876,0.002943,0.999849
4,0.000610,0.508434,0.001456,0.848749,0.001900,0.984875,0.002566,0.984875,0.002949,0.999849
...,...,...,...,...,...,...,...,...,...,...
115,0.002457,0.489625,0.003815,0.845621,0.004507,0.984562,0.005228,0.984562,0.005947,0.999846
116,0.002061,0.489337,0.003713,0.845534,0.004381,0.984553,0.005571,0.984553,0.006515,0.999846
117,0.001930,0.489866,0.003068,0.845694,0.003341,0.984569,0.004529,0.984569,0.005472,0.999846
118,0.002267,0.489718,0.002964,0.845650,0.003919,0.984565,0.004810,0.984565,0.004811,0.999846
